In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "8" 
os.environ["NUMEXPR_NUM_THREADS"] = "8" 
os.environ["OMP_NUM_THREADS"] = "8" 

from os import path, makedirs, listdir
import sys
import numpy as np
np.random.seed(1)
import random
random.seed(1)

import torch
from torch import nn
from torch.backends import cudnn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler

from tqdm import tqdm
import timeit
import cv2

from zoo.models import Dpn92_9ch_Unet, SeResNext50_Unet_9ch, Res34_9ch_Unet, Res50_9ch_Unet

from utils import *

from sklearn.model_selection import train_test_split

import ntpath

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)



/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# test_png = './wdata/AOI_10_Dar_Es_Salaam/test_png'
# test_png2 = './wdata/AOI_10_Dar_Es_Salaam/test_png_5_3_0'
# test_png3 = './wdata/AOI_10_Dar_Es_Salaam/test_png_pan_6_7'

# models_folder = './wdata/weights'
# output_folder = './wdata/AOI_10_Dar_Es_Salaam/test_pred'

test_png = './wdata/test_png'
test_png2 = './wdata/test_png_5_3_0'
test_png3 = './wdata/test_png_pan_6_7'

models_folder = './wdata/weights'
output_folder = './wdata/test_pred'


speed_bins = [15, 18.75, 20, 25, 30, 35, 45, 55, 65]


In [3]:
class TestData(Dataset):
    def __init__(self, files):
        super().__init__()
        self.files = files

    def __len__(self):
        return len(self.files) * 4

    def __getitem__(self, idx):
        fn = self.files[idx // 4]

        img_id = ntpath.basename(fn)[0:-4]

        img = cv2.imread(fn, cv2.IMREAD_COLOR)
        img2 = cv2.imread(fn.replace('test_png', 'test_png_5_3_0'), cv2.IMREAD_COLOR)
        img3 = cv2.imread(fn.replace('test_png', 'test_png_pan_6_7'), cv2.IMREAD_COLOR)

        img = np.pad(img, ((6, 6), (6, 6), (0, 0)), mode='reflect')
        img2 = np.pad(img2, ((6, 6), (6, 6), (0, 0)), mode='reflect')
        img3 = np.pad(img3, ((6, 6), (6, 6), (0, 0)), mode='reflect')

        img = np.concatenate([img, img2, img3], axis=2)

        if idx % 4 == 1:
            img = img[::-1, ...]
        elif idx % 4 == 2:
            img = img[:, ::-1, ...]
        elif idx % 4 == 3:
            img = img[::-1, ::-1, ...]

        img = preprocess_inputs(img)

        img = torch.from_numpy(img.transpose((2, 0, 1))).float()

        sample = {'img': img, 'img_id': img_id}
        return sample


In [4]:
# if __name__ == '__main__':
t0 = timeit.default_timer()

makedirs(output_folder, exist_ok=True)

vis_dev = '0,1,2,3'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = vis_dev

cudnn.benchmark = True

test_batch_size = 4 # 12

all_files = []
for f in listdir(test_png):
    if '.png' in f:
        all_files.append(path.join(test_png, f))

test_data = TestData(all_files)

test_data_loader = DataLoader(test_data, batch_size=test_batch_size, num_workers=4, shuffle=False)



In [8]:
len(listdir(test_png))

604

In [5]:
models = []

for fold in [3, 5, 7]:
    model = nn.DataParallel(SeResNext50_Unet_9ch(pretrained=None)).cuda()

    snap_to_load = 'seres50_9ch_{}_0_best'.format(fold)
    checkpoint = torch.load(path.join(models_folder, snap_to_load), map_location='cpu')
    loaded_dict = checkpoint['state_dict']
    sd = model.state_dict()
    for k in model.state_dict():
        if k in loaded_dict:
            sd[k] = loaded_dict[k]
    loaded_dict = sd
    model.load_state_dict(loaded_dict)
    print("loaded checkpoint '{}' (best_score {})"
            .format(snap_to_load, checkpoint['best_score']))

    model = model.eval()
    models.append(model)

for fold in [1,2,3]:
    model = nn.DataParallel(Res50_9ch_Unet(pretrained=False)).cuda()

    snap_to_load = 'res50_9ch_full_{}_0_best'.format(fold)
    checkpoint = torch.load(path.join(models_folder, snap_to_load), map_location='cpu')
    loaded_dict = checkpoint['state_dict']
    sd = model.state_dict()
    for k in model.state_dict():
        if k in loaded_dict:
            sd[k] = loaded_dict[k]
    loaded_dict = sd
    model.load_state_dict(loaded_dict)
    print("loaded checkpoint '{}' (best_score {})"
            .format(snap_to_load, checkpoint['best_score']))

    model = model.eval()
    models.append(model)



loaded checkpoint 'seres50_9ch_3_0_best' (best_score 0.6785936674078744)
loaded checkpoint 'seres50_9ch_5_0_best' (best_score 0.7054870851329355)
loaded checkpoint 'seres50_9ch_7_0_best' (best_score 0.6946809258987368)
loaded checkpoint 'res50_9ch_full_1_0_best' (best_score 0.6523791906505166)
loaded checkpoint 'res50_9ch_full_2_0_best' (best_score 0.6393917105118306)
loaded checkpoint 'res50_9ch_full_3_0_best' (best_score 0.6531817435357078)


In [6]:
with torch.no_grad():
    for sample in tqdm(test_data_loader):
        imgs = sample["img"].cuda(non_blocking=True)
        img_ids =  sample["img_id"]

        msk_preds = []
        speed_cont_preds = []
        msk_speed_preds = []
        ids = []
        for i in range(0, len(img_ids), 4):
            img_id = img_ids[i]
            ids.append(img_id)
            msk_preds.append([])
            speed_cont_preds.append([])
            msk_speed_preds.append([])

        for model in models:
            out = model(imgs)
            msk_pred0 = torch.sigmoid(out[:, :2, ...]).cpu().numpy()
            speed_cont_pred0 = out[:, 2, ...].cpu().numpy()
            speed_cont_pred0[speed_cont_pred0 < 0] = 0
            speed_cont_pred0[speed_cont_pred0 > 1] = 1
            msk_speed_pred0 = torch.softmax(out[:, 3:, ...], dim=1).cpu().numpy()

            for i in range(len(ids)):
                msk_preds[i].append(msk_pred0[i*4 + 0])
                msk_preds[i].append(msk_pred0[i*4 + 1, :, ::-1, :])
                msk_preds[i].append(msk_pred0[i*4 + 2, :, :, ::-1])
                msk_preds[i].append(msk_pred0[i*4 + 3, :, ::-1, ::-1])

                speed_cont_preds[i].append(speed_cont_pred0[i*4 + 0])
                speed_cont_preds[i].append(speed_cont_pred0[i*4 + 1, ::-1, :])
                speed_cont_preds[i].append(speed_cont_pred0[i*4 + 2, :, ::-1])
                speed_cont_preds[i].append(speed_cont_pred0[i*4 + 3, ::-1, ::-1])

                msk_speed_preds[i].append(msk_speed_pred0[i*4 + 0])
                msk_speed_preds[i].append(msk_speed_pred0[i*4 + 1, :, ::-1, :])
                msk_speed_preds[i].append(msk_speed_pred0[i*4 + 2, :, :, ::-1])
                msk_speed_preds[i].append(msk_speed_pred0[i*4 + 3, :, ::-1, ::-1])


        for i in range(len(ids)):
            msk_pred = np.asarray(msk_preds[i])
            msk_pred = msk_pred.mean(axis=0)
            speed_cont_pred = np.asarray(speed_cont_preds[i])
            speed_cont_pred = speed_cont_pred.mean(axis=0)
            msk_speed_pred = np.asarray(msk_speed_preds[i])
            msk_speed_pred = msk_speed_pred.mean(axis=0)

            pred_img0 = np.concatenate([msk_pred[0, ..., np.newaxis], msk_pred[1, ..., np.newaxis], np.zeros_like(msk_pred[0, ..., np.newaxis])], axis=2)
            cv2.imwrite(path.join(output_folder,  ids[i] + '.png'), (pred_img0 * 255).astype('uint8'), [cv2.IMWRITE_PNG_COMPRESSION, 9])
            cv2.imwrite(path.join(output_folder,  ids[i] + '_speed0.png'), (msk_speed_pred[:3].transpose(1, 2, 0) * 255).astype('uint8'), [cv2.IMWRITE_PNG_COMPRESSION, 9])
            cv2.imwrite(path.join(output_folder,  ids[i] + '_speed1.png'), (msk_speed_pred[3:6].transpose(1, 2, 0) * 255).astype('uint8'), [cv2.IMWRITE_PNG_COMPRESSION, 9])
            cv2.imwrite(path.join(output_folder,  ids[i] + '_speed2.png'), (msk_speed_pred[6:9].transpose(1, 2, 0) * 255).astype('uint8'), [cv2.IMWRITE_PNG_COMPRESSION, 9])
            cv2.imwrite(path.join(output_folder,  ids[i] + '_speed_cont.png'), (speed_cont_pred * 255).astype('uint8'), [cv2.IMWRITE_PNG_COMPRESSION, 9])

elapsed = timeit.default_timer() - t0
print('Time: {:.3f} min'.format(elapsed / 60)) 


100%|██████████| 604/604 [1:27:46<00:00,  8.72s/it]

Time: 87.982 min


In [49]:
imgs.shape

torch.Size([2, 9, 1312, 1312])

In [28]:
import matplotlib.pyplot as plt
plt.imshow(out[0,...].transpose((1,2,0)))

TypeError: transpose() received an invalid combination of arguments - got (tuple), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [32]:
out[0,...].transpose((1,2,0))

TypeError: transpose() received an invalid combination of arguments - got (tuple), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [4]:
checkpoint = torch.load('./wdata/weights/res50_9ch_full_7_0_best', map_location='cpu')
loaded_dict = checkpoint['state_dict']

In [14]:
model = nn.DataParallel(Res50_9ch_Unet(pretrained=False)).cuda()
model.state_dict()

OrderedDict([('module.conv6.layer.0.weight',
              tensor([[[[ 0.0013, -0.0098, -0.0120],
                        [ 0.0223,  0.0168,  0.0132],
                        [ 0.0091, -0.0093,  0.0171]],
              
                       [[ 0.0015, -0.0113, -0.0104],
                        [-0.0157, -0.0134, -0.0085],
                        [-0.0077,  0.0079, -0.0048]],
              
                       [[ 0.0103,  0.0059,  0.0016],
                        [ 0.0080, -0.0036,  0.0129],
                        [-0.0085, -0.0009,  0.0137]],
              
                       ...,
              
                       [[-0.0132,  0.0155, -0.0103],
                        [-0.0201,  0.0005,  0.0073],
                        [-0.0051, -0.0213, -0.0032]],
              
                       [[ 0.0036, -0.0052,  0.0102],
                        [ 0.0081, -0.0084, -0.0063],
                        [-0.0066, -0.0053, -0.0011]],
              
                       [[-0.0077,  0.

In [15]:
sd = model.state_dict()
for k in model.state_dict():
    if k in loaded_dict:
        sd[k] = loaded_dict[k]
loaded_dict = sd
model.load_state_dict(loaded_dict)


<All keys matched successfully>

In [17]:
model

DataParallel(
  (module): Res50_9ch_Unet(
    (conv6): ConvRelu(
      (layer): Sequential(
        (0): Conv2d(2048, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
    )
    (conv6_2): ConvRelu(
      (layer): Sequential(
        (0): Conv2d(1536, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
    )
    (conv7): ConvRelu(
      (layer): Sequential(
        (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
    )
    (conv7_2): ConvRelu(
      (layer): Sequential(
        (0): Conv2d(768, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
    )
    (conv8): ConvRelu(
      (layer): Sequential(
        (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
    )
    (conv8_2): ConvRelu(
      (layer): Sequential(
        (0): Co

In [52]:
test_batch_size = 8
all_files = []
for f in listdir('./wdata/train_png'):
    if '.png' in f:
        all_files.append(path.join(test_png, f))

test_data = TestData(all_files)

test_data_loader = DataLoader(test_data, batch_size=test_batch_size, num_workers=16, shuffle=False)


In [51]:
img_pad[:, ::-1,...]

array([[[ 94,  82,  72],
        [ 98,  85,  74],
        [ 98,  86,  75],
        ...,
        [126, 127, 134],
        [ 96,  96, 102],
        [ 61,  60,  64]],

       [[ 92,  80,  70],
        [ 93,  81,  71],
        [ 95,  83,  73],
        ...,
        [ 99,  99, 105],
        [ 95,  95, 100],
        [ 75,  74,  79]],

       [[ 96,  84,  75],
        [ 93,  82,  72],
        [ 92,  81,  71],
        ...,
        [ 99,  98, 104],
        [ 94,  94,  99],
        [ 75,  74,  78]],

       ...,

       [[216, 221, 228],
        [222, 228, 236],
        [217, 223, 232],
        ...,
        [ 96, 102, 112],
        [139, 147, 160],
        [201, 212, 230]],

       [[217, 223, 230],
        [216, 223, 231],
        [213, 220, 228],
        ...,
        [ 94,  99, 109],
        [117, 123, 134],
        [170, 180, 195]],

       [[215, 222, 230],
        [212, 219, 227],
        [211, 219, 227],
        ...,
        [112, 117, 128],
        [128, 134, 146],
        [163, 171, 185]]

In [58]:
idx = 1
img_id = 1
fn = './wdata/train_png/SN5_roads_train_AOI_7_Moscow_chip0.png'
img = cv2.imread(fn, cv2.IMREAD_COLOR)
img2 = cv2.imread(fn.replace('test_png', 'test_png_5_3_0'), cv2.IMREAD_COLOR)
img3 = cv2.imread(fn.replace('test_png', 'test_png_pan_6_7'), cv2.IMREAD_COLOR)

img = np.pad(img, ((6, 6), (6, 6), (0, 0)), mode='reflect')
img2 = np.pad(img2, ((6, 6), (6, 6), (0, 0)), mode='reflect')
img3 = np.pad(img3, ((6, 6), (6, 6), (0, 0)), mode='reflect')

img = np.concatenate([img, img2, img3], axis=2)

if idx % 4 == 1:
    img = img[::-1, ...]
elif idx % 4 == 2:
    img = img[:, ::-1, ...]
elif idx % 4 == 3:
    img = img[::-1, ::-1, ...]

img = preprocess_inputs(img)

img = torch.from_numpy(img.transpose((2, 0, 1))).float()

sample = {'img': img, 'img_id': img_id}


In [65]:
sample['img'].shape
img.shape, img2.shape,img3.shape

(torch.Size([9, 1312, 1312]), (1312, 1312, 3), (1312, 1312, 3))

In [68]:
img[:, :2, ...].shape

torch.Size([9, 2, 1312])

In [53]:
for sample in tqdm(test_data_loader):
    imgs = sample["img"].cuda(non_blocking)
    img_ids =  sample["img_id"]

  0%|          | 0/677 [00:00<?, ?it/s]Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x2aea30dd5630>>Exception ignored in: 
Traceback (most recent call last):
<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x2aea30dd5630>>Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x2aea30dd5630>>
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
Exception ignored in:     Traceback (most recent call last):

<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x2aea30dd5630>>Exception ignored in: Traceback (most recent call last):
self._shutdown_workers()  File "/users/PCON0003/



<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x2aea3e07f358>>      File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
self._shutdown_workers()
    
    Traceback (most recent call last):
assert self._parent_pid == os.getpid(), 'can only test a child process'  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
self._shutdown_workers()
Traceback (most recent call last):

self._shutdown_workers()  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    AssertionError    : can only test a child process
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/tor

    assert self._parent_pid == os.getpid(), 'can only test a child process'
  0%|          | 0/677 [00:18<?, ?it/s]can only test a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x2aea30dd55f8>>



ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-3-7a3cbd38ca95>", line 18, in __getitem__
    img = np.pad(img, ((6, 6), (6, 6), (0, 0)), mode='reflect')
  File "<__array_function__ internals>", line 6, in pad
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/numpy/lib/arraypad.py", line 746, in pad
    pad_width = _as_pairs(pad_width, array.ndim, as_index=Tru

Traceback (most recent call last):
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
Exception ignored in:     self._shutdown_workers()<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x2aea30dd5630>>

Traceback (most recent call last):
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
  File "/users/PCON0003/osu10670/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():    assert self._parent_pid == 

In [54]:
imgs

NameError: name 'imgs' is not defined